In [ ]:
!pip install -U sympy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.11.1
    Uninstalling sympy-1.11.1:
      Successfully uninstalled sympy-1.11.1


See how to use sympy to convert a system of first order equations into a single higher order equation so that Lebl's approach to the Laplace Transform can be applied. Also, see how to use the concepts from Activity 7a to apply the Laplace Transform directly on the original system with many fewer steps!


In [ ]:
import sympy as sp

In [ ]:
t=sp.symbols("t",real=True, positive=True)
s=sp.symbols("s")
y1=sp.Function("y1")
y2=sp.Function("y2")

In [ ]:
A = sp.Matrix([[-1,0],[1,-2]]) # solve the homogeneous system directly with the eigenvector method
P,D = A.diagonalize()
P,D

(Matrix([
 [0, 1],
 [1, 1]]),
 Matrix([
 [-2,  0],
 [ 0, -1]]))

In [ ]:
y0 = sp.Matrix([2,1]) # same initial conditions as we used in the Activity 6 example
y = P*sp.exp(D*t)*P.inv()*y0
y

Matrix([
[            2*exp(-t)],
[2*exp(-t) - exp(-2*t)]])

In [ ]:
yprime0 = A*y0 # check the initial value of y'
yprime0

Matrix([
[-2],
[ 0]])

In [ ]:
y1prime = y1(t).diff(t) # now define the equations of the system in sympy
y2prime = y2(t).diff(t)

In [ ]:
f1 = y1prime + y1(t)
f2 = y2prime - y1(t) + 2*y2(t)
f1

y1(t) + Derivative(y1(t), t)

In [ ]:
f2

-y1(t) + 2*y2(t) + Derivative(y2(t), t)

In [ ]:
f2prime = f2.diff(t)
f2prime

-Derivative(y1(t), t) + 2*Derivative(y2(t), t) + Derivative(y2(t), (t, 2))

In [ ]:
y1prime_eq1 = sp.solve(f1,y1(t).diff(t))[0]
y1prime_eq1

-y1(t)

In [ ]:
y1_eq2 = sp.solve(f2,y1(t))[0]
y1_eq2

2*y2(t) + Derivative(y2(t), t)

In [ ]:
f3 = f2prime.subs({y1prime:y1prime_eq1}).subs({y1(t):y1_eq2}) # substitute to get a single higher order equation
f3

2*y2(t) + 3*Derivative(y2(t), t) + Derivative(y2(t), (t, 2))

In [ ]:
Lf3 = sp.laplace_transform(f3,t,s,noconds=True) # take the Laplace Transform
Lf3

s**2*LaplaceTransform(y2(t), t, s) + 3*s*LaplaceTransform(y2(t), t, s) - s*y2(0) + 2*LaplaceTransform(y2(t), t, s) - 3*y2(0) - Subs(Derivative(y2(t), t), t, 0)

In [ ]:
Ly2 = sp.solve(Lf3,sp.LaplaceTransform(y2(t), t, s))[0] # solve for the Laplace Transform: Y2(s)
Ly2

(s*y2(0) + 3*y2(0) + Subs(Derivative(y2(t), t), t, 0))/(s**2 + 3*s + 2)

In [ ]:
Ly2_ic = Ly2.subs({y2(0):y0[1], y2prime.subs({t:0}):yprime0[1]}) # put in initial conditions, same as the example!
Ly2_ic

(s + 3)/(s**2 + 3*s + 2)

In [ ]:
SMA = sp.diag(s,s) - A # now use the Activity 7 approach. Much simpler!
SMA.inv()

Matrix([
[           1/(s + 1),         0],
[-1/(-s**2 - 3*s - 2), 1/(s + 2)]])

In [ ]:
Y = SMA.inv()*y0
Y

Matrix([
[                       2/(s + 1)],
[-2/(-s**2 - 3*s - 2) + 1/(s + 2)]])

In [ ]:
sp.simplify(Y[1]) # comes out the same in one simple step!

(s + 3)/(s**2 + 3*s + 2)

In [ ]:
sp.inverse_laplace_transform(Y[1],s,t) # now check against simple homogeneous solution.

2*exp(-t) - exp(-2*t)